In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '2,3'


from openprompt.data_utils import InputExample


import json
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict, Features, ClassLabel, Value

from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainerCallback
import evaluate
from datasets import load_metric
import numpy as np
import time




SEED = 42

df_data = pd.read_csv( '../new_data/CPED_A_with_role_1.tsv', sep = '\t')
df_data.head()

/home/wenzhy/wenzhy/ENTER/envs/wen/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/wenzhy/wenzhy/ENTER/envs/wen/lib/python3.9/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


,sent,labels,emotions,dialog_state,origin_sent,personality_description,affective_prompt
0,Speaker : what a coincidence; Speaker : the ca...,0,"['neutral', 'neutral', 'neutral', 'neutral']","[1, 1, 0, 1]",Speaker : what a coincidence; Speaker : the ca...,"Speaker is friendly, cooperative, empathetic, ...","The emotion of Speaker is initially neutral, S..."
1,Speaker : right; Others : Your kids go to scho...,1,"['neutral', 'neutral', 'neutral']","[1, 0, 1]",Speaker : right; Others : Your kids go to scho...,"Speaker is friendly, cooperative, empathetic, ...","The emotion of Speaker is initially neutral, t..."
2,Speaker : What a coincidence my son is here; S...,0,"['neutral', 'neutral', 'neutral', 'neutral']","[1, 1, 0, 1]",Speaker : What a coincidence my son is here; S...,"Speaker is friendly, cooperative, empathetic, ...","The emotion of Speaker is initially neutral, S..."
3,Others : class three; Speaker : My son is also...,0,"['neutral', 'neutral', 'neutral', 'neutral', '...","[0, 1, 1, 1, 0]",Others : class three; Speaker : My son is also...,"Speaker is friendly, cooperative, empathetic, ...","First, the emotion of others is neutral, Speak..."
4,"Speaker : Ji Yangyang, it seems that I heard F...",0,"['neutral', 'neutral', 'neutral', 'neutral', '...","[1, 1, 1, 0, 1]","Speaker : Ji Yangyang, it seems that I heard F...","Speaker is friendly, cooperative, empathetic, ...","The emotion of Speaker is initially neutral, S..."


In [2]:
def load_data(tsv_file):
    df = pd.read_csv(tsv_file, sep='\t')
    data_path = '../data/tmp.jsonl'
    json_data = df[['affective_prompt', 'personality_description', 'labels']].to_dict(orient="records")
    with open(data_path, 'w') as outfile:
        for row in json_data:
            json.dump(row, outfile)
            outfile.write('\n')

    class_names = ['no', 'yes']
    features = Features({'affective_prompt': Value('string'), 'personality_description': Value('string'), 'labels': ClassLabel(names=class_names)})
    dataset_dict = load_dataset("json", data_files=data_path, features=features)

    tmp_dict = dataset_dict['train'].train_test_split(test_size=0.2, shuffle=True, seed=SEED)
    train_dataset, remaining_dataset = tmp_dict['train'], tmp_dict['test']
    tmp_dict = remaining_dataset.train_test_split(test_size=0.5, shuffle=True, seed=SEED)
    valid_dataset, test_dataset = tmp_dict['train'], tmp_dict['test']
    dataset_dict = DatasetDict({
        'train': train_dataset,
        'validation': valid_dataset,
        'test': test_dataset
    })
    return dataset_dict

tsv_file = '../new_data/CPED_A_with_role_1.tsv'
dataset_dict = load_data(tsv_file)
dataset_dict

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/wenzhy/.cache/huggingface/datasets/json/default-b2f9f23a3df52b46/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['affective_prompt', 'personality_description', 'labels'],
        num_rows: 10611
    })
    validation: Dataset({
        features: ['affective_prompt', 'personality_description', 'labels'],
        num_rows: 1326
    })
    test: Dataset({
        features: ['affective_prompt', 'personality_description', 'labels'],
        num_rows: 1327
    })
})

In [3]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['affective_prompt', 'personality_description', 'labels'],
        num_rows: 10611
    })
    validation: Dataset({
        features: ['affective_prompt', 'personality_description', 'labels'],
        num_rows: 1326
    })
    test: Dataset({
        features: ['affective_prompt', 'personality_description', 'labels'],
        num_rows: 1327
    })
})

In [4]:
dataset = {}

for split in ['train', 'validation', 'test']:
    dataset[split] = []
    cnt = 0
    for data in dataset_dict[split]:
        input_example = InputExample(text_a = data['affective_prompt'], text_b = data['personality_description'], label=int(data['labels']), guid=cnt)
        dataset[split].append(input_example)
        cnt += 1
print(dataset['train'][0])

{
  "guid": 0,
  "label": 0,
  "meta": {},
  "text_a": "The emotion of Speaker is initially neutral, the emotion of others is neutral, Speaker respond with neutral, Speaker respond with neutral, the emotion of others is neutral, Speaker respond with neutral, the emotion of others is happy, Speaker respond with neutral, the emotion of others is happy, Speaker respond with neutral, the emotion of others is fear, ",
  "text_b": "Speaker is friendly, cooperative, empathetic, and compassionate, often prioritizing harmonious relationships and the well-being of others.",
  "tgt_text": null
}



In [5]:
from openprompt.plms import load_plm
plm, tokenizer, model_config, WrapperClass = load_plm("t5", "t5-base")

from openprompt.prompts import ManualTemplate
template_text = '{"placeholder":"text_a"} Question: {"placeholder":"text_b"}? Is it correct? {"mask"}.'
mytemplate = ManualTemplate(tokenizer=tokenizer, text=template_text)

# To better understand how does the template wrap the example, we visualize one instance.

wrapped_example = mytemplate.wrap_one_example(dataset['train'][0])
print(wrapped_example)

[[{'text': 'The emotion of Speaker is initially neutral, the emotion of others is neutral, Speaker respond with neutral, Speaker respond with neutral, the emotion of others is neutral, Speaker respond with neutral, the emotion of others is happy, Speaker respond with neutral, the emotion of others is happy, Speaker respond with neutral, the emotion of others is fear, ', 'loss_ids': 0, 'shortenable_ids': 1}, {'text': ' Question:', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': ' Speaker is friendly, cooperative, empathetic, and compassionate, often prioritizing harmonious relationships and the well-being of others.', 'loss_ids': 0, 'shortenable_ids': 1}, {'text': '? Is it correct?', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': '<mask>', 'loss_ids': 1, 'shortenable_ids': 0}, {'text': '.', 'loss_ids': 0, 'shortenable_ids': 0}], {'guid': 0, 'label': 0}]


/home/wenzhy/wenzhy/ENTER/envs/wen/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [6]:
from openprompt.plms import T5TokenizerWrapper
wrapped_t5tokenizer= T5TokenizerWrapper(max_seq_length=128, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")

# You can see what a tokenized example looks like by
tokenized_example = wrapped_t5tokenizer.tokenize_one_example(wrapped_example, teacher_forcing=False)
print(tokenized_example)
print(tokenizer.convert_ids_to_tokens(tokenized_example['input_ids']))
print(tokenizer.convert_ids_to_tokens(tokenized_example['decoder_input_ids']))

{'input_ids': [37, 13868, 13, 16778, 19, 7513, 7163, 6, 8, 13868, 13, 717, 19, 7163, 6, 16778, 3531, 28, 7163, 6, 16778, 3531, 28, 7163, 6, 8, 13868, 13, 717, 19, 7163, 6, 16778, 3531, 28, 7163, 6, 8, 13868, 13, 717, 19, 1095, 6, 16778, 3531, 28, 7163, 6, 8, 13868, 13, 717, 19, 1095, 6, 16778, 3531, 28, 7163, 6, 8, 13868, 13, 717, 19, 2971, 6, 11860, 10, 16778, 19, 2609, 6, 20270, 6, 3, 15, 51, 27826, 6, 11, 21801, 6, 557, 1884, 155, 2610, 29938, 3079, 11, 8, 168, 18, 9032, 13, 717, 5, 3, 58, 27, 7, 34, 2024, 58, 32099, 3, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'de

In [7]:
model_inputs = {}
for split in ['train', 'validation', 'test']:
    model_inputs[split] = []
    for sample in dataset[split]:
        tokenized_example = wrapped_t5tokenizer.tokenize_one_example(mytemplate.wrap_one_example(sample), teacher_forcing=False)
        model_inputs[split].append(tokenized_example)

In [8]:
from openprompt import PromptDataLoader

train_dataloader = PromptDataLoader(dataset=dataset["train"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=True, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")
# next(iter(train_dataloader))


tokenizing: 10611it [00:12, 854.91it/s]


In [9]:
from openprompt.prompts import ManualVerbalizer
import torch

# for example the verbalizer contains multiple label words in each class
myverbalizer = ManualVerbalizer(tokenizer, num_classes=2,
                        label_words=[["no"], ["yes"]])

print(myverbalizer.label_words_ids)
logits = torch.randn(2,len(tokenizer)) # creating a pseudo output from the plm, and
print(myverbalizer.process_logits(logits)) # see what the verbalizer do

Parameter containing:
tensor([[[ 150]],

        [[4273]]])
tensor([[-0.8573, -0.5521],
        [-0.1179, -2.1964]])


In [10]:
from openprompt import PromptForClassification

use_cuda = True
prompt_model = PromptForClassification(plm=plm,template=mytemplate, verbalizer=myverbalizer, freeze_plm=False)
if use_cuda:
    prompt_model=  prompt_model.cuda()

In [11]:
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
no_decay = ['bias', 'LayerNorm.weight']
# it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters = [
    {'params': [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)

for epoch in range(10):
    tot_loss = 0
    for step, inputs in enumerate(train_dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        logits = prompt_model(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        if step %100 ==1:
            print("Epoch {}, average loss: {}".format(epoch, tot_loss/(step+1)), flush=True)

# Evaluate
validation_dataloader = PromptDataLoader(dataset=dataset["validation"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

allpreds = []
alllabels = []
for step, inputs in enumerate(validation_dataloader):
    if use_cuda:
        inputs = inputs.cuda()
    logits = prompt_model(inputs)
    labels = inputs['label']
    alllabels.extend(labels.cpu().tolist())
    allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())

acc = sum([int(i==j) for i,j in zip(allpreds, alllabels)])/len(allpreds)
print(acc)

/home/wenzhy/wenzhy/ENTER/envs/wen/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 0, average loss: 0.9072587490081787
Epoch 0, average loss: 0.7479881766964408
Epoch 0, average loss: 0.7291503080194539
Epoch 0, average loss: 0.7233788271909518
Epoch 0, average loss: 0.7187242267366073
Epoch 0, average loss: 0.7120249651045913
Epoch 0, average loss: 0.7087310329079628
Epoch 0, average loss: 0.7077140286989362
Epoch 0, average loss: 0.7045512320729265
Epoch 0, average loss: 0.7038103512709263
Epoch 0, average loss: 0.7013836843138207
Epoch 0, average loss: 0.6993497412536841
Epoch 0, average loss: 0.6981281622897171
Epoch 0, average loss: 0.6975735885458791
Epoch 0, average loss: 0.6970583703289188
Epoch 0, average loss: 0.6963649274962402
Epoch 0, average loss: 0.6963506282473474
Epoch 0, average loss: 0.6950602769308869
Epoch 0, average loss: 0.6944451253286477
Epoch 0, average loss: 0.6942146129584964
Epoch 0, average loss: 0.6942017395551745
Epoch 0, average loss: 0.6939750889176193
Epoch 0, average loss: 0.6929506305520584
Epoch 0, average loss: 0.692558759

Epoch 7, average loss: 0.6565353347187737
Epoch 7, average loss: 0.6557840156502253
Epoch 7, average loss: 0.6559928118023085
Epoch 7, average loss: 0.6561078106487667
Epoch 7, average loss: 0.6557869559010362
Epoch 7, average loss: 0.6557227023805303
Epoch 7, average loss: 0.655284848552906
Epoch 7, average loss: 0.6561576699237839
Epoch 7, average loss: 0.6564231177480768
Epoch 7, average loss: 0.65611638693421
Epoch 8, average loss: 0.7444735765457153
Epoch 8, average loss: 0.6561670750379562
Epoch 8, average loss: 0.6447418644876763
Epoch 8, average loss: 0.645687034884036
Epoch 8, average loss: 0.6431140763249563
Epoch 8, average loss: 0.6378784012865735
Epoch 8, average loss: 0.6383996579635183
Epoch 8, average loss: 0.6395603914631058
Epoch 8, average loss: 0.6413290582764476
Epoch 8, average loss: 0.6404148774043948
Epoch 8, average loss: 0.6417409456121231
Epoch 8, average loss: 0.6432329580701631
Epoch 8, average loss: 0.6438298875995959
Epoch 8, average loss: 0.6456165798294

tokenizing: 1326it [00:01, 844.57it/s]


0.5942684766214178
